## 군집합 분석 -> k-평균 알고리짐(군집합)분석
### 샘플 데이터 : 온라인 판매 데이터(영국 특정 쇼핑몰의 샘플 데이터 기반)
### UCI 레포지토리 다운 받을 예정. 
### 분석 모델 구축 : 사이킷런의 내부에 있는 패키지를 이용하고 있는 중. 
### 구글, 메타, 파이썬의 내장 패키지등, 오픈 소스를 이용하고 있다. 

### 용어정리 
### 비지도 학습: 쉽게 말해서, 답이 없는 학습을 시켜서, 학습된 모델을 이용해서, 예측값 , 실제값 비교하는 
### 간단한 프로세스 
### x(훈련데이터)-> 머신러닝 프로그램 -> 학습 모델 -> 클러스터(군, 집합, 분류..)
###  새로운 데이터 -> 학습 모델 -> 예측 레이블(타겟, 결과등)

### 예) 쇼핑몰 , 소비 패턴을 파악하는 비지도학습 모델링. 
### 예2) 겨울에 1월 시즌에는 어는 물품이 많이 구매가 되었는지, 연령층 별로, 구매 패턴-> 분류를 
### 예3) 30 ~ 40대, 책 구매를 많이했고, 식료품에 다이어트 식품들이 구매가 많이 되었다. -> 광고,  

### k-평균 알고리즘 
### k개의 클러스터를 구성한다. k개의 중심점을 임의의 위치로 잡고 중심점을 기준으로 가까이 있는 데이터를
### 확인한 뒤, 그들과의 거리의 평균 지점을 중심점으로 이동한다. -> 나중에, 삼각형 모양으로 중심점 표기.
### 임의의 클러스터의 k개의 최적 갯수를 찾는 부분이 문제점
### 가장 좋은 k를 찾기 위한 방법으로, 엘보 방법, 실루엣 방법을 사용할 예정. 

### 엘보 방법. 
### 클러스터의 중심점과 클러스터 내의 데이터 거리 차이의 제곱값 합을 -> 왜곡이라고 부름 
### 선이 급격히 꺽이는 지점 있음. 그 지점 엘보(팔꿈치), 값을 최적의 k 값으로 사용함 -> 그래프로 꺽이는 지점, 보여드림. 

### 실루엣 분석
### 클러스터의 갯수를 얼마로 정할지를 지정해주는 분석 방법.
### 클러스터 내에 있는 데이터가 얼마나 조밀하게 모여 있는지를 측정하는 도구. 
### 데이터 i가 해당 클러스터(군, 집합)내의 데이터와 얼마나 가까운지를 나타내는 응집력 a(i)
### 가장 가까운 다른 클러스터 내의 데이터와 얼마나 떨어져 있는지 (분리도) b(i)
### 실루엣 계수 s(i)-> = (b(i) - a(i)) / max (a(i), b(i)) , 
### 실루엣 계수의 결괏값은 , -1 < s(i) <1, 결론, -> 1에 가까울수록 좋은 군집화. 


### 사이킷런에서, 함수로 다 제공해주니, 이용하기.



### 샘플 데이터 받기
### UCI 레포지토리에서 -> online retail 


In [1]:
# 엑셀 파일 읽기 도구 설치. 
!pip install openpyxl

In [2]:
import pandas as pd 
import math

# 영국 온라인 소매 플랫폼, 2010/12/1 , 2011/12/9 , 1년 동안 수집된 데이터. 
retail_df = pd.read_excel("./Online_Retail.xlsx")
retail_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


### Online_Retail.xlsx , 컬럼 기본 정보 
### InvoiceNo : 6자리 정소, 송장번호, C로 시작하는 것은 취소 주문 
### StockCode : 제퓸 고유 품목 코드 
### Description : 제품 설명 
### Quantity : 주문 수량 
### InvoiceDate : 주문 날짜 시간 
### UnitPrice : 제품 단가(영국 파운드화)
### CustomerID : 주문 고객 번호 
### Country : 주문 고객 국적

In [3]:
# 데이터 기본 정보를 확인. 
retail_df.info()
# 전체 데이터 갯수 : 541,909 개 
# Description :     540,455 개
# CustomerID  :     406,829 개

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [4]:
# 데이터 기본 정제
# 1 CustomerID 기존 : 실수형 -> 정수형 변경 
# 2 CustomerID 없는 데이터 제거 
# 3 Quantity, UnitPrice , 음수값 제거, 수량, 가격은 기본적으로 다 양이라서, 

# Quantity, UnitPrice , 음수값 제거,
retail_df = retail_df[retail_df["Quantity"]>0]
retail_df = retail_df[retail_df["UnitPrice"]>0]
# 2 CustomerID 없는 데이터 제거 
retail_df = retail_df[retail_df["CustomerID"].notnull()]

# 1 CustomerID 기존 : 실수형 -> 정수형 변경 
retail_df["CustomerID"] = retail_df["CustomerID"].astype(int)

# 확인, 
retail_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397884 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397884 non-null  object        
 1   StockCode    397884 non-null  object        
 2   Description  397884 non-null  object        
 3   Quantity     397884 non-null  int64         
 4   InvoiceDate  397884 non-null  datetime64[ns]
 5   UnitPrice    397884 non-null  float64       
 6   CustomerID   397884 non-null  int32         
 7   Country      397884 non-null  object        
dtypes: datetime64[ns](1), float64(1), int32(1), int64(1), object(4)
memory usage: 25.8+ MB


In [6]:
# 기본 null 값 여부 확인. null 값 모두 제거 확인. 
print(retail_df.isnull().sum())
# 차원으로 해당 데이터의 갯수 
print(retail_df.shape)

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64
(397884, 8)


In [7]:
# 중복 데이터, 레코드  제거
# inplace=True : 원본 데이터도 변경에 영향을 받겠다. 
retail_df.drop_duplicates(inplace=True)
print(retail_df.shape)

(392692, 8)


In [11]:
# 총 컬럼 8개 중에서, 필요한 컬럼만 가져오기
# 제품수, 거래 건수, 고객수 
# 수량 파악해보기. 
pd.DataFrame([
  {"Product":len(retail_df["StockCode"].value_counts()),
  "Transaction": len(retail_df["InvoiceNo"].value_counts()),
  "Customer": len(retail_df["CustomerID"].value_counts())}],
  columns = ["Product","Transaction","Customer"],
  index= ["counts"]
)


,Product,Transaction,Customer
counts,3665,18532,4338


In [12]:
# 국적 수량 파악
retail_df["Country"].value_counts()

United Kingdom          349203
Germany                   9025
France                    8326
EIRE                      7226
Spain                     2479
Netherlands               2359
Belgium                   2031
Switzerland               1841
Portugal                  1453
Australia                 1181
Norway                    1071
Italy                      758
Channel Islands            747
Finland                    685
Cyprus                     603
Sweden                     450
Austria                    398
Denmark                    380
Poland                     330
Japan                      321
Israel                     245
Unspecified                241
Singapore                  222
Iceland                    182
USA                        179
Canada                     151
Greece                     145
Malta                      112
United Arab Emirates        68
European Community          60
RSA                         57
Lebanon                     45
Lithuani

In [13]:
# 마케팅 분석 이용하기 위해서, 몇가지 데이터 더 추출하기. 
# 1 고객의 주문 횟수,
# 2 주문 총액 
# 3 마지막 주문 후 며칠이 지났는지 정보. 

# 새 컬럼 -> 주문 금액 추가하기. 주문 단가 * 주문 수량 = 총 주문 금액 
retail_df["SalesAmount"] = retail_df["UnitPrice"] * retail_df["Quantity"]

# 확인. 
retail_df.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,SalesAmount
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [14]:
# 집계 
# 송장 번호, 총 주문 금액, 주문 날짜를 , 집계 기준으로 사용.

# 송장 번호, 총 주문 금액, 주문 날짜를-> 하나의 딕션너리에 담아두기. 

aggregations = {
  "InvoiceNo": "count",
  "SalesAmount" : "sum",
  "InvoiceDate" : "max"
}

# CustomerID(구매자)를 기준으로 그룹을 나누고, 
# 위에 aggregations 이 기준으로 집계를 할 예정. 
customer_df = retail_df.groupby("CustomerID").agg(aggregations)
customer_df= customer_df.reset_index()
customer_df.head()

,CustomerID,InvoiceNo,SalesAmount,InvoiceDate
0,12346,1,77183.60,2011-01-18 10:01:00
1,12347,182,4310.00,2011-12-07 15:52:00
2,12348,31,1797.24,2011-09-25 13:13:00
3,12349,73,1757.55,2011-11-21 09:51:00
4,12350,17,334.40,2011-02-02 16:01:00


In [17]:
# 새컬럼 추가, Freq, ElapsedDays 추가. 
customer_df= customer_df.rename(columns={"InvoiceNo":"Freq", "InvoiceDate":"ElapsedDays"})
customer_df.head()

,CustomerID,Freq,SalesAmount,ElapsedDays
0,12346,1,77183.60,2011-01-18 10:01:00
1,12347,182,4310.00,2011-12-07 15:52:00
2,12348,31,1797.24,2011-09-25 13:13:00
3,12349,73,1757.55,2011-11-21 09:51:00
4,12350,17,334.40,2011-02-02 16:01:00


In [18]:
# 마지막 주문일로 부터 얼마나 시간이 지났는지 여부를 확인.
# 기준 날짜는 임의로 : 2011년 12월 10일 
# 기준 날짜 - 마지막 구매일 : 일수로 계산.

import datetime

# 기준 날짜 - 마지막 구매일 : 일수로 계산해서, 새 컬럼에 추가. 
customer_df["ElapsedDays"]= datetime.datetime(2011,12,10) - customer_df["ElapsedDays"]
customer_df.head()


,CustomerID,Freq,SalesAmount,ElapsedDays
0,12346,1,77183.60,325 days 13:59:00
1,12347,182,4310.00,2 days 08:08:00
2,12348,31,1797.24,75 days 10:47:00
3,12349,73,1757.55,18 days 14:09:00
4,12350,17,334.40,310 days 07:59:00
